In [10]:
#import data
import csv
import os
path = 'Test1'
with open('images2.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    i=0
    for row in csvReader:
        if i==0:
            i=2
            continue
        filename=row[0]
        filepath=os.path.join(path,filename)
        all_files=os.listdir(filepath)
        for x in all_files:
            newfilename=x.split(".")[0]+"_"+row[1]+"_"+row[2]+"."+x.split(".")[1]
            newfilename_path=os.path.join(filepath,newfilename)
            oldfilename_path=os.path.join(filepath,x)
            os.rename(oldfilename_path,newfilename_path)

In [56]:
import numpy as np
import cv2
import os
import glob

X_data = []
dir_files ="Test1/imgaes/"
for img in os.listdir(dir_files):
        img_array = cv2.imread(os.path.join(dir_files,img), cv2.IMREAD_GRAYSCALE)
        resized_image = cv2.resize(img_array, (28, 28))
#         img_flatten = resized_image.reshape(-1)
        X_data.append(resized_image)
np_X_data = np.array(X_data)        
        
print(np_X_data.shape)

(38, 28, 28)


In [97]:
from PIL import Image
import os, sys
import glob
import numpy as np

Y_data = []
#root_dir = "Dissertation/list_of_images/"
#for ii in glob.iglob(root_dir + '**/*.jpg', recursive=True):
dir_files = glob.glob("Test1/imgaes/**.JPG")
for ii in dir_files:
# #     print(ii)
#     im = Image.open(ii)
#     imResize = im.resize((28,28), Image.ANTIALIAS)
#     aaa=np.array(imResize)
#     X_data.append(aaa)
    if ii.find('_Not Corroded_Not Damaged')!=-1:
        Y_data.append(2)
    elif ii.find('_Not Corroded_Damaged')!=-1:
        Y_data.append(1)
    elif ii.find('_Corroded_Not Damaged')!=-1:
        Y_data.append(3)
    elif ii.find('_Corroded_Damaged')!=-1:
        Y_data.append(0)

np_Y_data = np.array(Y_data) 
# print(len(X_data))

In [98]:
print(len(np_Y_data))

38


In [99]:
print(np_X_data[0,:,:].shape)
# print(np_X_data)

(28, 28)


In [100]:
# N = 1 # how much data of each class
c1, c2, c3= 2, 1, 0

In [101]:
index_of_c1 = np.argwhere(np_Y_data == c1).ravel() # index of 2s
index_of_c1 = np.random.choice(index_of_c1, replace=False)

index_of_c2 = np.argwhere(np_Y_data == c2).ravel() # index of 3s
index_of_c2 = np.random.choice(index_of_c2, replace=False)

index_of_c3 = np.argwhere(np_Y_data == c3).ravel() # index of 4s
index_of_c3 = np.random.choice(index_of_c3, replace=False)

# index_of_c4 = np.argwhere(np_Y_data == c4).ravel() # index of 4s
# index_of_c4 = np.random.choice(index_of_c4, N, replace=False)

x_train = np_X_data[np.hstack((index_of_c1, index_of_c2, index_of_c3))] # subsample
y_train = np_Y_data[np.hstack((index_of_c1, index_of_c2, index_of_c3))] # subsample

print("size of training subset: %s" % str(x_train.shape))

size of training subset: (3, 28, 28)


In [102]:
print(y_train.shape)

(3,)


In [103]:
as_rgb = lambda im: np.tile(np.expand_dims(im,2),(1,1,3))

#example
print(as_rgb(x_train[0,:,:]).shape)

(28, 28, 3)


In [104]:
import skimage
resize = lambda im: skimage.transform.resize(as_rgb(im), (299, 299), preserve_range=False, order=0)

new_x_train = np.concatenate([[resize(np_X_data[i,:,:])] for i in range(np_X_data.shape[0])])
print(new_x_train.shape)

(38, 299, 299, 3)


In [105]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [106]:
inception = InceptionV3(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(299,299,3)
)

In [107]:
training_features = inception.predict(new_x_train)
print(training_features.shape)

(38, 2048)


In [108]:
from sklearn.svm import LinearSVC as Classifier

In [110]:
clf = Classifier()

clf.fit(training_features, np_Y_data)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [111]:
y_predict = clf.predict(training_features)

In [112]:
acc = clf.score(training_features, np_Y_data)
print("accuracy: %f" % acc)
print("successfully classified: %d/%d" % (np.sum(y_predict == np_Y_data), len(np_Y_data)))

accuracy: 1.000000
successfully classified: 38/38


In [115]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

classifiers = {
    'svm': SVC, 
    'sgd': SGDClassifier, 
    'linearsvm': LinearSVC,
    'decisiontree': DecisionTreeClassifier,
    'naivebayes': GaussianNB
}

In [116]:
for name, Classifer in classifiers.items():
    clf = Classifer()
    clf.fit(training_features, np_Y_data)
    train_acc = clf.score(training_features, np_Y_data)
    test_acc  = clf.score(training_features,  np_Y_data)
    print("classifer: %s\n  training accuracy: %f\n  test accuracy: %f" % (name, train_acc, test_acc))

classifer: svm
  training accuracy: 0.421053
  test accuracy: 0.421053
classifer: sgd
  training accuracy: 1.000000
  test accuracy: 1.000000
classifer: linearsvm
  training accuracy: 1.000000
  test accuracy: 1.000000
classifer: decisiontree
  training accuracy: 1.000000
  test accuracy: 1.000000
classifer: naivebayes
  training accuracy: 1.000000
  test accuracy: 1.000000


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
